# Automating the API

Now that we have successfully run one query, we can run automate things. This means we can make work that would otherwise be very bulky much easier. 

Say, for example, we wanted to find out what records were held at TNA for every ship in the Royal Navy during the war of 1812? By hand, this would require a huge amount of browsing. With automation, this can be made a lot easier. 

The first thing we need is a list of ship names. Helpfully, this is [available on Wikipedia](https://en.wikipedia.org/wiki/Category:War_of_1812_ships_of_the_United_Kingdom). The full list is shown in [the aditional data file](./aditional_data.py). Here, we are going to show just the first 3 ships. 
Note that, to simplify the code, we are using the names of ships with an HMS prefix, and only one name. 

In [ ]:
three_ships = [
    "Devastation",
    "Dreadnought",
    "Invincible"
]

With some research, we can see that TNA has provided guides on how to search teh archives. In particular, they have provided information on [how to find logs and records from Royal Navy ships](https://www.nationalarchives.gov.uk/help-with-your-research/research-guides/royal-navy-ships-voyages-log-books/). From this, we can create another list - the various record series we would want to search. 

From the API documentation we saw before, we can see that these record series have to be included in a specific format. As with the list of ship names, we've included the full list in [the aditional data file](./aditional_data.py). Here, we are going to show just 3 record series.

In [ ]:
"sps.recordSeries=ADM%2055", # record_series_ADM_55
"sps.recordSeries=ADM%20101", # record_series_ADM_101
"sps.recordSeries=MT%2032" # record_series_MT_32

With two lists, we can start to automate the requests. 

We will start by importing the libraries we need, along with the full length lists. 

In [ ]:
%pip install requests
import requests

import aditional_data

ship_list = aditional_data.ships

record_series = aditional_data.admiralty_record_series

ship_data = []

As we are going to automate the requests, we are going to need a place to store the results. We are going to use a list of list of dictionaries, making our results both easy to read and easy to work with. 

From the discovery API documentation, we can decide what bits of information we want. We are going to use the following:
- The ship name
- The record start date
- The record end date
- The record ID
- The record description
- Whether the record is digitised

It is often helpful to write an example of the results you want. It helps to visualise your goal, making it easier to build code. An example of the results we are looking for is below. 

In [ ]:
[
    {
        "ship-name": "Devastation",
        "records" : [
            {
                "startDate": "1871-01-01",
                "endDate": "1871-12-31",
                "id": "C1234567",
                "description": "A description of the record",
                "disgitised": True
            },
            {
                "startDate": "1872-01-01",
                "endDate": "1872-12-31",
                "id": "C1234568",
                "description": "A description of the record",
                "disgitised": True
            }
        ]
    }, ## These outer brackets, and all data within, are then repeated for each ship
]

So, now we have a list of ships, a list of record series, and a list of the information we want. We can start to build our code.

This next cell is where where the requests actually happen - hence, there are line-by-line comments to give a detailed explanation of what is happening. The general aim of this cell is to show the power of an API and automation. Here, instead of having to search for each ship by hand, we can loop through the list and search for each ship automatically.

There are two further bits of information we need to note. 
1 - We are going to artificially limit the searches to the first 10 entries in the list. This is to prevent the Discovery API from rejecting our requests. From the [help page](https://www.nationalarchives.gov.uk/help/discovery-for-developers-about-the-application-programming-interface-api/), there is a request for less than 3,000 requests a day, and a rate of approx 1/sec. These limits are often in place on API's to prevent from overloading the servers - if several people were to seperately run all the requests in this notebook, an accidental [DDoS](https://en.wikipedia.org/wiki/Denial-of-service_attack) could occur. This is not going to happen with the requests in this notebook, but it is good practice to limit the number of requests you make, and for servers to limit the number of requests they accept.

Another approach to this would be to slow down the requests - you could wait for a set period of time between each request by using the [time.sleep()](https://docs.python.org/3/library/time.html#time.sleep) function. 

2 - We are going to need to do these requests in two steps. From the documentation, we can see that start and end date, and id are all avaialable from the general search, btu the description and digitisation status are not. To get these, we need to make a second request to the [`/records/details/{id}` endpoint](https://discovery.nationalarchives.gov.uk/API/sandbox/index#/Records). These requests will need to be for individual records, and will provide the extra information we need.

In [ ]:
for ship in ship_list:                                                                  # We start by looping through each ship in the list
        url = "https://discovery.nationalarchives.gov.uk/API/search/records?"           # For every ship, we need to build a custom URL
        for record in record_series:                                                    # We need to add each record series we want to search within, so we loop through the list of record series
                url += record                                                           # We add the record series to the URL
                url += "&"                                                              # An ampersand is added to seperate query parameters
        url += "q=" + ship                                                              # We add the query parameter to the URL - this is the ship name 
        headers = {
                'Accept': 'application/json'                                            # Setting the headers indicates to the API that we want to receive JSON data. This will make it easier to work with the response data
            }
        response = requests.request("GET", url, headers=headers)                        # THe request is actually made! We store the response in a variable called response
        response_json = response.json()                 
        if response_json["records"] != []:
                records_found = []
                for record in response_json["records"]:
                        records_found.append(
                                {
                                        "id": record["id"],
                                        "startDate": record["startDate"],
                                        "endDate": record["endDate"]
                                }
                        )
                ship_data.append(
                        {
                                "ship-name": ship,
                                "records": records_found
                        }
                )

print(ship_data)